In [2]:
import os

def list_files(startpath):
    # Folders to skip to keep output readable
    skip_dirs = {'.git', '.dvc', 'venv311', '__pycache__', '.ipynb_checkpoints'}
    
    for root, dirs, files in os.walk(startpath):
        # Filter out skipped directories
        dirs[:] = [d for d in dirs if d not in skip_dirs]
        
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print(f'{subindent}{f}')

if __name__ == "__main__":
    print(f"Current Structure for: {os.getcwd()}\n")
    list_files('..')

Current Structure for: c:\Users\Admin\Desktop\ResumeRecommenderMLops\notebooks

../
    .dvcignore
    .env
    .gitignore
    dvc.lock
    dvc.yaml
    params.yaml
    pyproject.toml
    README.md
    requirements.txt
    status.txt
    .github/
        workflows/
            serpapi_data_ingestion.yml
    app/
    artifacts/
    data/
        constants/
            jobs.csv
            locations.yaml
        processed/
        raw/
            serpapi/
                2026-01/
                    AI_Engineer_Bengaluru_India_page41.json
                    AI_Engineer_Chennai_India_page44.json
                    AI_Engineer_Delhi_India_page45.json
                    AI_Engineer_Hyderabad_India_page42.json
                    AI_Engineer_Pune_India_page43.json
                    Analytics_Engineer_Bengaluru_India_page83.json
                    Analytics_Engineer_Chennai_India_page86.json
                    Analytics_Engineer_Hyderabad_India_page84.json
                    Analytic

In [1]:
print("hi")

hi


In [2]:
#basic path test 
import sys
import os
from pathlib import Path
import pandas as pd

# 1. Fix the path logic so the Notebook can see the 'utils' folder
# This assumes your notebook is in a subfolder. If it's in the root, use "."
sys.path.append(os.path.abspath("..")) 

try:
    from utils.paths import (
        BASE_DIR, 
        RAW_SERPAPI_DIR, 
        PROCESSED_SERPAPI_DIR, 
        LOG_DIR, 
        JOBS_CSV_PATH, 
        LOCATIONS_YAML_PATH, 
        PARAMS_PATH
    )

    # 2. Define the paths to check
    paths_to_check = {
        "Project Root (BASE_DIR)": BASE_DIR,
        "Raw Data Folder": RAW_SERPAPI_DIR,
        "Processed Data Folder": PROCESSED_SERPAPI_DIR,
        "Logs Folder": LOG_DIR,
        "Jobs CSV File": JOBS_CSV_PATH,
        "Locations YAML File": LOCATIONS_YAML_PATH,
        "Global paramas file ": PARAMS_PATH
    }

    # 3. Perform Validation
    results = []
    for name, path in paths_to_check.items():
        results.append({
            "Component": name,
            "Exists": "✅ Yes" if path.exists() else "❌ NO",
            "Type": "Directory" if path.is_dir() else "File" if path.is_file() else "Missing",
            "Absolute Path": str(path.resolve())
        })

    # 4. Display as a nice Table
    df_results = pd.DataFrame(results)
    display(df_results)

    # 5. Summary Warning
    missing_count = sum(1 for r in results if "❌" in r["Exists"])
    if missing_count > 0:
        print(f"\n⚠️ WARNING: {missing_count} path(s) are invalid. Check your BASE_DIR in utils/paths.py!")
    else:
        print("\n🚀 ALL SYSTEM PATHS ARE VALID!")

except ModuleNotFoundError:
    print("❌ ERROR: Could not find 'utils' folder. Ensure you are running the notebook from the correct directory.")

,Component,Exists,Type,Absolute Path
0,Project Root (BASE_DIR),✅ Yes,Directory,C:\Users\Admin\Desktop\ResumeRecommenderMLops
1,Raw Data Folder,✅ Yes,Directory,C:\Users\Admin\Desktop\ResumeRecommenderMLops\...
2,Processed Data Folder,✅ Yes,Directory,C:\Users\Admin\Desktop\ResumeRecommenderMLops\...
3,Logs Folder,✅ Yes,Directory,C:\Users\Admin\Desktop\ResumeRecommenderMLops\...
4,Jobs CSV File,✅ Yes,File,C:\Users\Admin\Desktop\ResumeRecommenderMLops\...
5,Locations YAML File,✅ Yes,File,C:\Users\Admin\Desktop\ResumeRecommenderMLops\...
6,Global paramas file,✅ Yes,File,C:\Users\Admin\Desktop\ResumeRecommenderMLops\...



🚀 ALL SYSTEM PATHS ARE VALID!


In [1]:
import sys
import os
from pathlib import Path
import pandas as pd

# 1. Fix the path logic so the Notebook can see the 'utils' folder
# This assumes your notebook is in a subfolder. If it's in the root, use "."
sys.path.append(os.path.abspath("..")) 


from utils.paths import (
        BASE_DIR, 
        RAW_SERPAPI_DIR, 
        PROCESSED_SERPAPI_DIR, 
        LOG_DIR, 
        JOBS_CSV_PATH, 
        LOCATIONS_YAML_PATH
)

print(BASE_DIR)


C:\Users\Admin\Desktop\ResumeRecommenderMLops


In [22]:
import pandas as pd
from utils.paths import get_processed_data_path
from utils.dates import current_run_month

# 1. Load your latest processed data
csv_path = get_processed_data_path(current_run_month())
df = pd.read_csv(csv_path)

# 2. Define the placeholders we used in process_data.py
placeholders = ["Unknown", "Not mentioned", "None"]

# 3. Create the report
report_data = []

for col in df.columns:
    # Standard NaNs
    nan_count = df[col].isnull().sum()
    
    # Placeholder counts (Unknown, etc.)
    placeholder_count = df[col].astype(str).isin(placeholders).sum()
    
    total_missing = nan_count + placeholder_count
    
    report_data.append({
        "Column": col,
        "Actual NaN": nan_count,
        "Placeholders": placeholder_count,
        "Total Missing": total_missing,
        "Usability %": round(((len(df) - total_missing) / len(df)) * 100, 2)
    })

# 4. Display as a clean DataFrame
quality_report = pd.DataFrame(report_data).set_index("Column")

print(f"--- Data Quality Heartbeat: {current_run_month()} ---")
print(quality_report.to_string())

--- Data Quality Heartbeat: 2026-01 ---
                     Actual NaN  Placeholders  Total Missing  Usability %
Column                                                                   
job_id                        0             0              0       100.00
title                         0             0              0       100.00
company_name                  0             0              0       100.00
location                      0             1              1        99.89
description                   0             0              0       100.00
salary                        0           835            835         8.24
schedule_type                 0             1              1        99.89
posted_at                     0           292            292        67.91
search_term                   0             0              0       100.00
search_location               0             0              0       100.00
ingestion_month               0             0              0       100.0

In [23]:
df.sample(5)

,job_id,title,company_name,location,description,salary,schedule_type,posted_at,search_term,search_location,ingestion_month,ingestion_timestamp,data_source,search_engine,llm_based_category,job_link,apply_link
259,eyJqb2JfdGl0bGUiOiJEYXRhIFNjaWVudGlzdCBJSUkgLS...,Data Scientist III - Pricing Analytics,Expedia Group,"Gurugram, Haryana",Expedia Group brands power global travel for e...,Not mentioned,Full–time,3 days ago,Data Scientist,"Gurugram, India",2026-01,2026-01-22 19:08:02 UTC,serpapi,google_jobs,Unknown,https://www.google.com/search?ibp=htl;jobs&q=D...,https://in.linkedin.com/jobs/view/data-scienti...
567,eyJqb2JfdGl0bGUiOiJQcmFjdGljZSBNYW5hZ2VyIOKAky...,Practice Manager – Data Engineering,Lagozon Technologies Pvt. Ltd,"New Delhi, Delhi","As a Practice Manager in Data Engineering, you...",Not mentioned,Full–time,Unknown,Data Engineer,"Noida, India",2026-01,2026-01-22 19:11:44 UTC,serpapi,google_jobs,Unknown,https://www.google.com/search?ibp=htl;jobs&q=D...,https://www.lagozon.com/job/practice-manager-d...
425,eyJqb2JfdGl0bGUiOiJHZW5lcmF0aXZlIEFJIEVuZ2luZW...,Generative AI Engineer,Turing,"Gandhinagar, Gujarat",About the role: Turing is looking for people w...,Not mentioned,Full–time,10 days ago,Generative AI Engineer,"Ahmedabad, India",2026-01,2026-01-22 19:09:54 UTC,serpapi,google_jobs,Unknown,https://www.google.com/search?ibp=htl;jobs&q=G...,https://getmereferred.com/job-listing/generati...
866,eyJqb2JfdGl0bGUiOiJTdGFmZiBCYWNrZW5kIEVuZ2luZW...,Staff Backend Engineer,Tide,"New Delhi, Delhi",ABOUT TIDE At Tide we help SMEs save time (and...,Not mentioned,Full–time,7 days ago,Backend Engineer,"New Delhi, India",2026-01,2026-01-22 19:06:38 UTC,serpapi,google_jobs,Unknown,https://www.google.com/search?ibp=htl;jobs&q=B...,https://builtindelhi.in/job/staff-backend-engi...
156,eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QgSW50ZXJuIi...,Data Analyst Intern,Opkey,"Noida, Uttar Pradesh",Opkey | Series B Funded | Noida (In-Office) | ...,Not mentioned,Internship,21 days ago,Data Analyst,"Noida, India",2026-01,2026-01-22 19:11:04 UTC,serpapi,google_jobs,Unknown,https://www.google.com/search?ibp=htl;jobs&q=D...,https://nextleap.app/jobs/data-analyst/data-an...
